# Chapter 72: Interactive Exploration Tools

## Learning Objectives

By the end of this chapter, you will be able to:

- Understand the role of interactive exploration in understanding time‑series data and models
- Use Jupyter widgets (ipywidgets) to create interactive controls for exploring NEPSE data
- Build parameter tuning interfaces to adjust model hyperparameters and see immediate effects
- Implement what‑if analysis tools that allow users to change input features and observe prediction changes
- Create model comparison dashboards to evaluate multiple models side‑by‑side
- Develop scenario planning tools to simulate different market conditions
- Enable ad‑hoc querying of predictions and features using interactive filters
- Export results and visualizations for reports and presentations
- Apply best practices for designing intuitive and responsive interactive tools

---

## Introduction

Static visualizations, while useful, have limitations: they show a single view of the data. **Interactive exploration tools** empower users to ask their own questions, drill down into details, and test hypotheses in real time. For the NEPSE prediction system, interactive tools can help traders, analysts, and data scientists:

- Explore how predictions change with different input features.
- Tune model hyperparameters and immediately see the impact on performance.
- Compare multiple models to select the best one for deployment.
- Simulate what happens if market conditions change (e.g., a sudden spike in volume).
- Query historical predictions for specific stocks or time periods.

In this chapter, we will build interactive tools using **Jupyter widgets (ipywidgets)** and integrate them with visualizations. We'll also discuss how to package these tools into standalone applications using **Voilà**. By the end, you'll be able to create rich, interactive experiences that make your NEPSE system more accessible and insightful.

---

## 72.1 Jupyter Widgets (ipywidgets)

Jupyter widgets are interactive HTML widgets for Jupyter notebooks. They provide sliders, dropdowns, buttons, and more, which can be linked to Python code. When combined with plotting libraries, they create powerful interactive dashboards directly in the notebook.

### 72.1.1 Installation

```bash
pip install ipywidgets
jupyter nbextension enable --py widgetsnbextension
```

For JupyterLab, you may need additional extensions; but for simplicity, we'll work in the classic notebook.

### 72.1.2 Basic Widgets

Let's create a simple interactive plot that allows the user to select a stock symbol and see its closing price.

```python
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
import plotly.express as px

# Load data (assume we have a DataFrame with columns for each stock)
df_prices = pd.read_csv('nepse_prices.csv', index_col=0, parse_dates=True)

# Create a dropdown for stock selection
stock_dropdown = widgets.Dropdown(
    options=df_prices.columns,
    value=df_prices.columns[0],
    description='Stock:',
    disabled=False
)

# Create an output widget to hold the plot
output = widgets.Output()

# Define the update function
def update_plot(change):
    with output:
        output.clear_output(wait=True)  # clear previous plot
        selected = stock_dropdown.value
        fig = px.line(df_prices, x=df_prices.index, y=selected, title=f'{selected} Closing Price')
        fig.show()

# Attach the update function to the dropdown's value change
stock_dropdown.observe(update_plot, names='value')

# Display widgets
display(stock_dropdown, output)

# Call once to initialize
update_plot(None)
```

**Explanation:**  
- `widgets.Dropdown` creates a dropdown menu with stock symbols.
- `widgets.Output` is a placeholder where we will render the plot.
- The `observe` method calls `update_plot` whenever the dropdown value changes.
- Inside `update_plot`, we clear the output and draw a new Plotly figure.
- The final `display` shows the widgets.

This simple example demonstrates the core pattern: widgets → callback → update visualization.

---

## 72.2 Parameter Tuning Interfaces

When developing models, data scientists often need to experiment with hyperparameters. An interactive interface can speed up this process.

### 72.2.1 Interactive Hyperparameter Tuning

Let's build a tool that trains a Random Forest model on NEPSE data with adjustable hyperparameters and displays the resulting accuracy.

```python
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load feature-engineered data (assume we have X and y already)
# For demo, we'll create synthetic data
np.random.seed(42)
X = np.random.randn(1000, 10)
y = (X[:, 0] + X[:, 1] > 0).astype(int)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create widgets
n_estimators_slider = widgets.IntSlider(value=100, min=10, max=500, step=10, description='n_estimators')
max_depth_slider = widgets.IntSlider(value=10, min=1, max=50, step=1, description='max_depth')
min_samples_split_slider = widgets.IntSlider(value=2, min=2, max=20, step=1, description='min_samples_split')
train_button = widgets.Button(description='Train Model')
output = widgets.Output()

def train_model(b):
    with output:
        output.clear_output()
        # Get current parameter values
        n_est = n_estimators_slider.value
        max_d = max_depth_slider.value
        min_split = min_samples_split_slider.value
        
        # Train model
        model = RandomForestClassifier(
            n_estimators=n_est,
            max_depth=max_d,
            min_samples_split=min_split,
            random_state=42,
            n_jobs=-1
        )
        model.fit(X_train, y_train)
        
        # Evaluate
        y_pred = model.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        
        print(f"Accuracy: {acc:.4f}")
        print(f"Parameters: n_estimators={n_est}, max_depth={max_d}, min_samples_split={min_split}")

train_button.on_click(train_model)

# Display widgets
display(n_estimators_slider, max_depth_slider, min_samples_split_slider, train_button, output)
```

**Explanation:**  
- Sliders allow the user to choose hyperparameters.
- A button triggers the training.
- The callback trains the model and prints the accuracy.
- This interactive loop lets the user quickly explore the parameter space.

### 72.2.2 Live Feature Importance Plot

We can extend this to show feature importance after each training.

```python
def train_model(b):
    with output:
        output.clear_output()
        # ... train model as before ...
        
        # Feature importance
        importances = model.feature_importances_
        features = [f'Feature {i}' for i in range(X.shape[1])]
        fig = px.bar(x=features, y=importances, title='Feature Importance')
        fig.show()
```

Now the user sees not only accuracy but also which features matter most.

---

## 72.3 What‑If Analysis

What‑if analysis allows users to change input features and see how the prediction changes. This is particularly useful for understanding model behaviour and building trust.

### 72.3.1 Single Instance What‑If

We'll create a tool where the user selects a stock (or a specific instance) and adjusts feature values via sliders, then sees the model's prediction.

```python
# Assume we have a pre-trained model and a scaler
import joblib
model = joblib.load('nepse_xgboost.pkl')
scaler = joblib.load('scaler.pkl')
feature_names = ['SMA_20', 'RSI', 'Volume', 'Lag1', 'Lag2']  # example

# Create sliders for each feature
sliders = {}
for name in feature_names:
    sliders[name] = widgets.FloatSlider(value=0.0, min=-3, max=3, step=0.1, description=name)

predict_button = widgets.Button(description='Predict')
output = widgets.Output()

def predict(b):
    with output:
        output.clear_output()
        # Collect feature values
        features = np.array([[sliders[name].value for name in feature_names]])
        # Scale
        features_scaled = scaler.transform(features)
        # Predict
        prob = model.predict_proba(features_scaled)[0, 1]
        print(f"Predicted probability of upward move: {prob:.3f}")

predict_button.on_click(predict)

# Display all sliders and button
display(*sliders.values(), predict_button, output)
```

**Explanation:**  
- Each feature gets a slider with a reasonable range (here we assume scaled features, so range -3 to 3).
- The user adjusts sliders and clicks "Predict" to see the output.
- This allows exploration of the model's decision boundary.

### 72.3.2 What‑If with Real Data

We can also let the user select an actual historical instance as a starting point, then tweak features.

```python
# Load a sample of actual data
sample_df = pd.read_csv('nepse_sample.csv')
index_selector = widgets.Dropdown(
    options=list(range(len(sample_df))),
    description='Sample Index:'
)

def load_sample(change):
    idx = index_selector.value
    row = sample_df.iloc[idx]
    for name in feature_names:
        if name in row:
            sliders[name].value = row[name]  # set slider to actual value

index_selector.observe(load_sample, names='value')

display(index_selector, *sliders.values(), predict_button, output)
```

**Explanation:**  
- A dropdown lets the user pick a historical sample.
- When selected, the sliders are set to that sample's feature values.
- The user can then modify sliders to see how predictions change.

---

## 72.4 Model Comparison Tools

When multiple candidate models exist, an interactive comparison tool helps select the best one.

### 72.4.1 Side‑by‑Side Metrics

We'll create a tool that displays key metrics for several pre‑trained models.

```python
# Assume we have a dictionary of models and their test metrics
models = {
    'Random Forest': {'accuracy': 0.62, 'precision': 0.64, 'recall': 0.59},
    'XGBoost': {'accuracy': 0.65, 'precision': 0.66, 'recall': 0.63},
    'LSTM': {'accuracy': 0.63, 'precision': 0.65, 'recall': 0.60}
}

model_selector = widgets.Dropdown(
    options=list(models.keys()),
    description='Model:'
)

output = widgets.Output()

def show_metrics(change):
    with output:
        output.clear_output()
        selected = model_selector.value
        metrics = models[selected]
        print(f"Metrics for {selected}:")
        for k, v in metrics.items():
            print(f"  {k}: {v:.3f}")

model_selector.observe(show_metrics, names='value')

display(model_selector, output)
```

### 72.4.2 Visual Comparison

We can also plot performance metrics as bar charts.

```python
import matplotlib.pyplot as plt

def plot_metrics(change):
    with output:
        output.clear_output(wait=True)
        selected = model_selector.value
        metrics = models[selected]
        
        fig, ax = plt.subplots()
        ax.bar(metrics.keys(), metrics.values())
        ax.set_ylim(0, 1)
        ax.set_ylabel('Score')
        ax.set_title(f'Metrics for {selected}')
        plt.show()

model_selector.observe(plot_metrics, names='value')
```

For a more comprehensive comparison, you could show all models at once using grouped bar charts.

---

## 72.5 Scenario Planning

Scenario planning tools allow users to simulate hypothetical market conditions and see the model's response.

### 72.5.1 Simulating Market Shocks

Suppose we want to see how a sudden increase in volatility affects predictions. We can create a tool that scales the volatility feature.

```python
# Assume we have a function that takes modified features and returns predictions
def scenario_prediction(volatility_multiplier, volume_multiplier):
    # Get baseline features from a recent instance
    base_features = sample_df.iloc[-1][feature_names].copy()
    # Modify volatility-related features (e.g., RSI might be affected)
    # This is a simplistic example; in reality, you'd need a proper simulator
    modified = base_features.copy()
    modified['RSI'] = min(100, max(0, modified['RSI'] * volatility_multiplier))
    modified['Volume'] *= volume_multiplier
    # Scale and predict
    features_scaled = scaler.transform([modified])
    prob = model.predict_proba(features_scaled)[0,1]
    return prob

vol_slider = widgets.FloatSlider(value=1.0, min=0.5, max=2.0, step=0.1, description='Volatility Mult')
vol_slider_vol = widgets.FloatSlider(value=1.0, min=0.5, max=2.0, step=0.1, description='Volume Mult')
scenario_output = widgets.Output()

def update_scenario(change):
    with scenario_output:
        scenario_output.clear_output()
        prob = scenario_prediction(vol_slider.value, vol_slider_vol.value)
        print(f"Predicted probability with multipliers: {prob:.3f}")

vol_slider.observe(update_scenario, names='value')
vol_slider_vol.observe(update_scenario, names='value')

display(vol_slider, vol_slider_vol, scenario_output)
```

**Explanation:**  
- The sliders control multipliers for volatility and volume.
- The prediction updates in real time as sliders move (if we use `continuous_update=True` or observe).
- This lets users explore "what if" scenarios.

---

## 72.6 Ad‑Hoc Querying

Users may want to query predictions for specific stocks or date ranges. Interactive filters make this easy.

### 72.6.1 Filterable Data Table

We can use `ipywidgets` to filter a DataFrame and display the results.

```python
from IPython.display import display_html

# Assume we have a DataFrame of predictions with columns: date, symbol, prediction, actual
df_pred = pd.read_csv('nepse_predictions.csv', parse_dates=['date'])

# Create widgets
symbol_selector = widgets.SelectMultiple(
    options=df_pred['symbol'].unique(),
    value=[df_pred['symbol'].unique()[0]],
    description='Symbols'
)

date_range = widgets.DatePicker(
    description='Start Date',
    value=df_pred['date'].min()
)
date_range_end = widgets.DatePicker(
    description='End Date',
    value=df_pred['date'].max()
)

filter_button = widgets.Button(description='Apply Filters')
output = widgets.Output()

def filter_data(b):
    with output:
        output.clear_output()
        # Apply filters
        mask = (df_pred['symbol'].isin(symbol_selector.value)) & \
               (df_pred['date'] >= date_range.value) & \
               (df_pred['date'] <= date_range_end.value)
        filtered = df_pred[mask]
        display_html(filtered.to_html(), raw=True)

filter_button.on_click(filter_data)

display(symbol_selector, date_range, date_range_end, filter_button, output)
```

**Explanation:**  
- `SelectMultiple` allows choosing multiple symbols.
- Date pickers set the date range.
- Clicking the button applies the filters and displays the filtered DataFrame as HTML.

### 72.6.2 Interactive Query with SQL

If your data is in a database, you could use widgets to build SQL queries interactively. This is more advanced but follows the same pattern.

---

## 72.7 Export Functionality

Interactive tools are more useful if users can save results. We can add export buttons to download data or plots.

### 72.7.1 Export Filtered Data to CSV

```python
import io
import base64

export_button = widgets.Button(description='Export to CSV')
export_output = widgets.Output()

def export_data(b):
    with export_output:
        export_output.clear_output()
        # Get current filtered data (reuse filter logic)
        mask = (df_pred['symbol'].isin(symbol_selector.value)) & \
               (df_pred['date'] >= date_range.value) & \
               (df_pred['date'] <= date_range_end.value)
        filtered = df_pred[mask]
        csv = filtered.to_csv(index=False)
        b64 = base64.b64encode(csv.encode()).decode()
        href = f'<a download="filtered_predictions.csv" href="data:text/csv;base64,{b64}">Download CSV</a>'
        display_html(href, raw=True)

export_button.on_click(export_data)
```

**Explanation:**  
- The button triggers generation of a CSV from the current filtered data.
- The CSV is encoded as base64 and presented as a downloadable link.

### 72.7.2 Export Current Plot

For Plotly figures, you can use the `plotly.io.write_image` function to save as PNG, but in a notebook you might provide a download link. However, Plotly figures have a built‑in camera icon for export; you may not need to implement custom export.

---

## 72.8 Packaging with Voilà

Voilà turns Jupyter notebooks into standalone web applications. It hides the code cells and shows only the widgets and outputs.

### 72.8.1 Preparing a Notebook for Voilà

1. Ensure all widgets and outputs are at the top level (not inside functions that aren't called).
2. Use `display` to show widgets.
3. Avoid using `input()` or other blocking calls.
4. Test the notebook by running all cells.

### 72.8.2 Running Voilà

```bash
voila my_notebook.ipynb
```

This starts a local server and opens the app in a browser.

### 72.8.3 Deploying Voilà Apps

You can deploy Voilà apps on:

- **Binder**: Share a link that launches a temporary instance.
- **Heroku**: Use a `Procfile` with `web: voila --port=$PORT my_notebook.ipynb`.
- **JupyterHub**: If you have JupyterHub, Voilà can be integrated.
- **Cloud VM**: Run as a service with systemd.

Voilà apps are a great way to share interactive tools with non‑technical users.

---

## 72.9 Best Practices for Interactive Tools

1. **Keep it responsive**: Avoid long computations in callbacks. If an operation is slow, add a loading indicator.
2. **Provide clear labels**: Describe what each widget does.
3. **Set reasonable defaults**: Users should see something useful immediately.
4. **Handle errors gracefully**: If a widget value leads to an error, show a helpful message.
5. **Use layout containers**: Group related widgets using `widgets.HBox`, `VBox`, or `Tab` to organise the interface.
6. **Document the tool**: Include a brief description of how to use it.
7. **Test on different browsers**: Ensure compatibility.

---

## Chapter Summary

In this chapter, we explored interactive exploration tools for the NEPSE prediction system. We covered:

- Using Jupyter widgets (ipywidgets) to create sliders, dropdowns, buttons, and output areas.
- Building parameter tuning interfaces for hyperparameter experimentation.
- Implementing what‑if analysis to understand model behaviour.
- Creating model comparison tools to evaluate multiple models.
- Developing scenario planning tools to simulate market conditions.
- Adding ad‑hoc querying capabilities with interactive filters.
- Providing export functionality for data and visualizations.
- Packaging interactive notebooks as standalone apps with Voilà.

Interactive tools bridge the gap between static analysis and user‑driven exploration. They empower traders, analysts, and data scientists to gain deeper insights from the NEPSE prediction system. In the next chapter, we will discuss **Alerting and Notification Systems**, which ensure that users are promptly informed of important events.

---

**End of Chapter 72**